In [128]:
from skmob.tessellation import tilers
import numpy as np
import pandas as pd
import pickle
import skmob
from shapely.geometry import Point


In [129]:
CACHE = True

In [2]:
import geopandas as gpd
url_tess = skmob.utils.constants.NY_COUNTIES_2011
tessellation = tilers.tiler.get("squared", base_shape="New York City, New York", meters=1500)

In [3]:
FILENAME = "utils/2013-12 - Citi Bike trip data.csv"

In [4]:
for i, row in tessellation.iterrows():
    if row['geometry'].contains(Point(-73.989978, 40.720196)):
        print(i)

322


In [5]:
df = pd.read_csv(FILENAME, sep=',')
tdf = skmob.TrajDataFrame(df, user_id='user', datetime='datetime')
tdf.head()

,tripduration,starttime,stoptime,start station id,start station name,start station latitude,start station longitude,end station id,end station name,end station latitude,end station longitude,bikeid,usertype,birth year,gender
0,1015,2013-12-01 00:00:03,2013-12-01 00:16:58,401,Allen St & Rivington St,40.720196,-73.989978,476,E 31 St & 3 Ave,40.743943,-73.979661,14729,Subscriber,1979,2
1,962,2013-12-01 00:00:07,2013-12-01 00:16:09,312,Allen St & E Houston St,40.722055,-73.989111,223,W 13 St & 7 Ave,40.737815,-73.999947,17871,Subscriber,1970,2
2,768,2013-12-01 00:00:15,2013-12-01 00:13:03,326,E 11 St & 1 Ave,40.729538,-73.984267,291,Madison St & Montgomery St,40.713126,-73.984844,18508,Subscriber,1982,2
3,218,2013-12-01 00:00:49,2013-12-01 00:04:27,2021,W 45 St & 8 Ave,40.759291,-73.988597,447,8 Ave & W 52 St,40.763707,-73.985162,17745,Subscriber,1976,1
4,550,2013-12-01 00:01:28,2013-12-01 00:10:38,439,E 4 St & 2 Ave,40.726281,-73.989780,296,Division St & Bowery,40.714131,-73.997047,16947,Subscriber,1983,1


In [6]:
tdf['origin'] = df.apply(lambda row: Point(row['start station longitude'], row['start station latitude']), axis=1)
tdf['destination'] = df.apply(lambda row: Point(row['end station longitude'], row['end station latitude']), axis=1)

In [7]:
def getIndexPosition(tessellation, position):
    for i, row in enumerate(tessellation):
        if row.contains(position):
            return i

In [8]:
origins = tdf['origin']
poligons = tessellation['geometry']

In [9]:
lista = []
for i, el in enumerate(origins):
    lista.append(getIndexPosition(poligons, el))



In [10]:
tdf['origin'] = lista

In [11]:
tdf['destination'] = tdf.apply(lambda row: getIndexPosition(poligons, row['destination']), axis=1)

In [12]:
tdf.columns

Index(['tripduration', 'starttime', 'stoptime', 'start station id',
       'start station name', 'start station latitude',
       'start station longitude', 'end station id', 'end station name',
       'end station latitude', 'end station longitude', 'bikeid', 'usertype',
       'birth year', 'gender', 'origin', 'destination'],
      dtype='object')

In [13]:
fdf = skmob.FlowDataFrame(tdf,
                        tessellation=tessellation,
                        origin="origin",
                        destination="destination",
                        tile_id='tile_ID')

In [14]:
fdf['flow'] = 1

In [130]:
if CACHE:
    with open('fdf.pkl', 'wb') as f:
        pickle.dump(fdf, f, protocol=pickle.HIGHEST_PROTOCOL)

In [15]:
fdf

,tripduration,starttime,stoptime,start station id,start station name,start station latitude,start station longitude,end station id,end station name,end station latitude,end station longitude,bikeid,usertype,birth year,gender,origin,destination,flow
0,1015,2013-12-01 00:00:03,2013-12-01 00:16:58,401,Allen St & Rivington St,40.720196,-73.989978,476,E 31 St & 3 Ave,40.743943,-73.979661,14729,Subscriber,1979,2,322,352,1
1,962,2013-12-01 00:00:07,2013-12-01 00:16:09,312,Allen St & E Houston St,40.722055,-73.989111,223,W 13 St & 7 Ave,40.737815,-73.999947,17871,Subscriber,1970,2,349,324,1
2,768,2013-12-01 00:00:15,2013-12-01 00:13:03,326,E 11 St & 1 Ave,40.729538,-73.984267,291,Madison St & Montgomery St,40.713126,-73.984844,18508,Subscriber,1982,2,350,349,1
3,218,2013-12-01 00:00:49,2013-12-01 00:04:27,2021,W 45 St & 8 Ave,40.759291,-73.988597,447,8 Ave & W 52 St,40.763707,-73.985162,17745,Subscriber,1976,1,353,353,1
4,550,2013-12-01 00:01:28,2013-12-01 00:10:38,439,E 4 St & 2 Ave,40.726281,-73.989780,296,Division St & Bowery,40.714131,-73.997047,16947,Subscriber,1983,1,323,322,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
443961,1014,2013-12-31 23:55:56,2014-01-01 00:12:50,350,Clinton St & Grand St,40.715595,-73.987030,532,S 5 Pl & S 4 St,40.710451,-73.960876,16336,Subscriber,1973,1,349,410,1
443962,682,2013-12-31 23:57:04,2014-01-01 00:08:26,538,W 49 St & 5 Ave,40.757952,-73.977876,2006,Central Park S & 6 Ave,40.765909,-73.976342,18352,Subscriber,1986,1,353,354,1
443963,1520,2013-12-31 23:57:44,2014-01-01 00:23:04,281,Grand Army Plaza & Central Park S,40.764397,-73.973715,411,E 6 St & Avenue D,40.722281,-73.976687,15000,Subscriber,1985,1,384,349,1
443964,1166,2013-12-31 23:58:14,2014-01-01 00:17:40,492,W 33 St & 7 Ave,40.750200,-73.990931,253,W 13 St & 5 Ave,40.735439,-73.994539,21022,Subscriber,1955,1,325,324,1


In [22]:
flow_df = fdf[['starttime','origin','destination', 'flow']]

In [23]:
flow_df['starttime'] = pd.to_datetime(df.starttime)

In [53]:
print(flow_df.min())
print(flow_df.max())

starttime      2013-12-01 00:00:03
origin                         271
destination                    271
flow                             1
dtype: object
starttime      2013-12-31 23:58:16
origin                         415
destination                    415
flow                             1
dtype: object


In [68]:
flow_df['origin'] = flow_df['origin'].astype(int)-271

In [69]:
flow_df['destination'] = flow_df['destination'].astype(int)-271

In [103]:
print(flow_df.min())
print(flow_df.max())

starttime      2013-12-01 00:00:03
origin                           0
destination                      0
flow                             1
dtype: object
starttime      2013-12-31 23:58:16
origin                         144
destination                    144
flow                             1
dtype: object


In [132]:
f = flow_df.groupby([pd.Grouper(key='starttime', freq='60min'),'origin','destination']).sum()
f

flow
starttime           origin destination      
2013-12-01 00:00:00 24     51              1
                           78              1
                    25     24              1
                           25              1
                           27              1
...                                      ...
2013-12-31 23:00:00 113    81              1
                    137    105             1
                    138    136             1
                    140    79              1
                           137             1

[152432 rows x 1 columns]

In [99]:
# TODO ridefinire
# origin_dest = np.zeros([int(flow_df['origin'].max())-int(flow_df['origin'].min()), int(flow_df['destination'].max())-int(flow_df['destination'].min())])

In [100]:
np.shape(origin_dest)

(144, 144)

In [101]:
for i, el in f.iterrows():
    print(el.name)
    break

(Timestamp('2013-12-01 00:00:00'), 24, 51)


In [124]:
time = set()
time_orgin_dest = []
origin_dest = np.zeros(
            [int(flow_df['origin'].max())+1-int(flow_df['origin'].min()), 
            int(flow_df['destination'].max())+1-int(flow_df['destination'].min())])
flow = list(f['flow'])

for i, el in enumerate(f.index):
    if(el[0] not in time):
        time.add(el[0])
        time_orgin_dest.append(origin_dest)
        origin_dest = np.zeros(
            [int(flow_df['origin'].max())+1-int(flow_df['origin'].min()), 
            int(flow_df['destination'].max())+1-int(flow_df['destination'].min())])
    origin_dest[el[1],el[2]] = flow[i]

In [125]:
np.shape(time_orgin_dest)

(744, 145, 145)

In [117]:
np.append([1, 2, 3], [[4, 5, 6], [7, 8, 9]])

array([1, 2, 3, 4, 5, 6, 7, 8, 9])